# Core 1.3 MLRun Basics - MLRun Projects

In this section, we will cover the MLRun Projects and how they can be used to structure and execute code

---

### References

Much of the following content is derived from the official documenation:
- [Projects](https://docs.mlrun.org/en/latest/projects/project.html)

---

### MLRun Projects

A project is a container for all your work on a particular activity/application. It is the basic starting point for your work. Project definitions include all of the associated code, functions, Submitting tasks/jobs to functions, artifacts, lists of parameters, and secrets.

MLRun Projects make it very simple to deploy to production via checking out a Git repo and immediately runing an automated pipeline. Because the project includes the underlying source code and build configuration, the same environment can be recreated.

---

### Create a Project

Let's create our first `MLRun Project`. We can create one via the UI or Python SDK. For this guide, we will use the Python SDK. You must specify a `name` and `context` directory (i.e. where the project will be located) like so:

```python
import mlrun

project = mlrun.get_or_create_project(name="my_project", context="./")
```

If a project of that name already exists it will be retrieved, otherwise a new project will be created

---

### Add Elements to the Project

Once the project exists, you can add things to it such as MLRun Functions and Workflows. The details of Functions and Workflows will be covered in later sections. The syntax for adding elements looks like the following:
```python
project.set_function(name='prep-data', func='prep_data.py', image='mlrun/mlrun', handler='prep_data')
project.set_function(name='train', func='hub://sklearn_classifier')
project.set_workflow(name='main', workflow_path="myflow.py")
```

You may notice that the `set_function` capabilities look very similar to `code_to_function` - and you would be correct. The `set_function` syntax can act as a simpler version of `code_to_function`, but they can also be used together like so:
```python
import mlrun

fn = mlrun.code_to_function(
    name="prep-data",
    project="my_project,
    filename="prep_data.py",
    image="mlrun/mlrun,
    handler="prep_data"
)

project.set_function(name='prep-data', func=fn)
```

### Save the Project

Finally, the project is saved via the following:
```python
project.save()
```

This creates a `project.yaml` file - the source of truth for the project that can be stored/shared via version control. When executing functions or workflows, MLRun will look to the `project.yaml` file to find the required source code.

The `project.yaml` file for the above will look like:
```yaml
kind: project
metadata:
  name: my_project
spec:
  functions:
  - url: prep_data.py
    name: prep-data
    image: mlrun/mlrun
    handler: prep_data
  - url: hub://sklearn_classifier
    name: train
  workflows:
  - name: main
    path: myflow.py
    engine: kfp
```

---

### Run a Function

Once everything is added to the project, running an individual function is very simple. It would look like:
```python
import mlrun

project = mlrun.get_or_create_project(name="my_project", context="./")

fn = project.get_function("prep-data")
fn.run()
```

---

### Run a Workflow

One of the best uses for a project is executing a workflow (pipeline) in an automated fashion. MLRun projects make it very simple to check out a Git repo and immediately run a pipeline via the Python SDK or CLI.

Running via the Python SDK will look like:
```python
import mlrun

project = mlrun.get_or_create_project(name="my_project", context="./")
project.run("main", arguments={'data': 'https://s3.wasabisys.com/iguazio/data/iris/iris.data.raw.csv'})
```

Running via the CLI will look like:
```bash
mlrun project --run main --arguments data=https://s3.wasabisys.com/iguazio/data/iris/iris.data.raw.csv
```

---